In [1]:
import os
import yt_dlp
from dotenv import load_dotenv

load_dotenv('./.env')

True

In [ ]:
def download_mp4_from_youtube(url):
    # Set the options for the download
    filename = 'infoslack.mp4'
    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]',
        'outtmpl': filename,
        'quiet': True,
    }

    # Download the video file
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        result = ydl.extract_info(url, download=True)

url = "https://www.youtube.com/watch?v=iw2TeYESnTk"
download_mp4_from_youtube(url)

In [ ]:
def download_audio(url):
    
    filename = 'infoslack-audio'
    ydl_opts = {
        'format': 'bestaudio',
        'postprocessors':[{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '128',
        }],
        'outtmpl': filename,
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        result = ydl.extract_info(url, download=True)
        
download_audio("https://www.youtube.com/watch?v=iw2TeYESnTk")

In [2]:
%%time
# whisper local
import whisper
model = whisper.load_model("base")
result = model.transcribe("infoslack-audio.mp3")

/Users/infoslack/Projects/yt_br_llm/envs/lib/python3.12/site-packages/whisper/__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_

CPU times: user 6min 13s, sys: 1min 31s, total: 7min 44s
Wall time: 1min 1s


In [3]:
result['text']

' O orden bearings são a forma de representar palavras e textos como vetores numericos. E a coisa mais interessante sobre esse conceito é que quando você pega palavras ou frases e converse uma representação numérica, palavras que são numericamente semelhantes são semelhantes em significado também. E isso nos permite construir coisas como mecanismo de busca, com um nível de precisão interessante. Nesse vídeo vamos explorar um sistema de busca semânticos usando emberes da OpenAI. Então, como funciona esse mecanismo de busca? Como pegar essas palavras e frases, convertem númeres para realizar a classificação, da textão de anomalias, agrupamento, enfim, todas essas tarefas legais de linguagem natural. Bem, eu preparei esse exemplo sobre emberes de palavras e agora vamos ver como tudo isso funciona. A matemática por trás, de uma maneira que você possa entender e aplicar. Eu vou deixar um link desse projeto na descrição. Bem, a primeira coisa que precisamos fazer é instalar o pacote da OpenA

In [4]:
%%time
# whisper API
from openai import OpenAI
client = OpenAI()

audio_file= open("infoslack-audio.mp3", "rb")
transcript = client.audio.transcriptions.create(
    model="whisper-1", 
    file=audio_file,
    response_format="text"
)

CPU times: user 213 ms, sys: 73.6 ms, total: 286 ms
Wall time: 41 s


In [5]:
with open ('text.txt', 'w') as file:
    file.write(transcript)

#### output openai

In [6]:
transcript

'Word Embeddings são uma forma de representar palavras e textos como vetores numéricos. E a coisa mais interessante sobre esse conceito é que quando você pega palavras ou frases e converte em uma representação numérica, palavras que são numericamente semelhantes são semelhantes em significado também. E isso nos permite construir coisas como um mecanismo de busca com um nível de precisão interessante. Nesse vídeo vamos explorar um sistema de busca semântico usando Embeddings da OpenAI. Então, como funciona esse mecanismo de busca? Como pegar essas palavras e frases, converter em números para realizar classificação, detecção de anomalias, agrupamento, enfim, todas essas tarefas legais de linguagem natural? Bem, eu preparei esse exemplo sobre Embeddings de palavras e agora vamos ver como tudo isso funciona. Há matemática por trás, de uma maneira que você possa entender e aplicar. Eu vou deixar um link desse projeto na descrição. Bem, a primeira coisa que precisamos fazer é instalar o paco

In [7]:
%%time
# whisper API Groq
from groq import Groq
client = Groq()

resp = client.audio.transcriptions.create(
    model="whisper-large-v3",
    file=audio_file,
    response_format="text"
)

CPU times: user 141 ms, sys: 64.5 ms, total: 205 ms
Wall time: 14.8 s


#### output groq api

In [8]:
resp

' Word Embeddings são a forma de representar palavras e textos como vetores numéricos. E a coisa mais interessante sobre esse conceito é que quando você pega palavras ou frases e converte em uma representação numérica, palavras que são numericamente semelhantes são semelhantes em significado também. E isso nos permite construir coisas como um mecanismo de busca com um nível de precisão interessante. Nesse vídeo vamos explorar um sistema de busca semântico usando Embeddings da OpenAI. Então, como funciona esse mecanismo de busca? Como pegar essas palavras e frases, converter em números para realizar a classificação, detecção de anomalias, agrupamento, enfim, todas essas tarefas legais de linguagem natural. Bem, eu preparei esse exemplo sobre embeddings de palavras, e agora vamos ver como tudo isso funciona. Há matemática por trás, de uma maneira que você possa entender e aplicar. Eu vou deixar um link desse projeto na descrição. Bem, a primeira coisa que precisamos fazer é instalar o pa

## Resumo

In [15]:
from langchain import OpenAI
from langchain_openai import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [16]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=0, separators=[" ", ",", "\n"]
)

In [17]:
from langchain.docstore.document import Document

with open('text.txt') as f:
    text = f.read()

texts = text_splitter.split_text(text)
docs = [Document(page_content=t) for t in texts]

In [18]:
docs[0]

Document(page_content='Word Embeddings são uma forma de representar palavras e textos como vetores numéricos. E a coisa mais interessante sobre esse conceito é que quando você pega palavras ou frases e converte em uma representação numérica, palavras que são numericamente semelhantes são semelhantes em significado também. E isso nos permite construir coisas como um mecanismo de busca com um nível de precisão interessante. Nesse vídeo vamos explorar um sistema de busca semântico usando Embeddings da OpenAI. Então, como funciona esse mecanismo de busca? Como pegar essas palavras e frases, converter em números para realizar classificação, detecção de anomalias, agrupamento, enfim, todas essas tarefas legais de linguagem natural? Bem, eu preparei esse exemplo sobre Embeddings de palavras e agora vamos ver como tudo isso funciona. Há matemática por trás, de uma maneira que você possa entender e aplicar. Eu vou deixar um link desse projeto na descrição. Bem, a primeira coisa que precisamos f

In [19]:
# prompt
prompt_template = """Escreva um resumo conciso com bullet points do texto abaixo:


{text}


RESUMO CONCISO:"""

BULLET_POINT_PROMPT = PromptTemplate(template=prompt_template, 
                        input_variables=["text"])

In [21]:
# summarization chai
llm_chain = LLMChain(llm=llm, prompt=BULLET_POINT_PROMPT)
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="text")
stuff_chain.invoke(docs)["output_text"]

'- **Word Embeddings**: Representação de palavras e textos como vetores numéricos, onde palavras semelhantes têm representações numéricas próximas.\n- **Mecanismo de Busca Semântico**: Utiliza Embeddings da OpenAI para melhorar a precisão em buscas.\n- **Processo de Criação de Embeddings**:\n  - Instalação do pacote OpenAI e importação de bibliotecas como pandas e numpy.\n  - Configuração da chave de API da OpenAI.\n  - Conversão de palavras em vetores usando a função `getEmbedding`.\n- **Cálculo de Similaridade**:\n  - Utilização da similaridade do cosseno para medir a proximidade entre vetores.\n  - Aplicação da função de similaridade em um DataFrame para encontrar palavras semelhantes.\n- **Desenvolvimento de Assistentes Inteligentes**: Necessidade de entender significados e intenções em consultas, superando limitações de buscas tradicionais.\n- **Uso de Vector Databases**: Armazenamento de embeddings de frases ou parágrafos para expandir a base de conhecimento de chatbots.\n- **Exe

## RAG

In [22]:
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

url = os.getenv("QDRANT_HOST")
api_key = os.getenv("QDRANT_API_KEY")

qdrant = Qdrant.from_documents(
    documents=docs,
    embedding=embeddings,
    url=url,
    api_key=api_key,
    collection_name="whisper"
)

In [23]:
# similarity test
query = "o que são word embeddings?"
qdrant.similarity_search(query)

[Document(metadata={'_id': '074d1cbc-614f-4e3d-add9-4e16da33f945', '_collection_name': 'whisper'}, page_content='Word Embeddings são uma forma de representar palavras e textos como vetores numéricos. E a coisa mais interessante sobre esse conceito é que quando você pega palavras ou frases e converte em uma representação numérica, palavras que são numericamente semelhantes são semelhantes em significado também. E isso nos permite construir coisas como um mecanismo de busca com um nível de precisão interessante. Nesse vídeo vamos explorar um sistema de busca semântico usando Embeddings da OpenAI. Então, como funciona esse mecanismo de busca? Como pegar essas palavras e frases, converter em números para realizar classificação, detecção de anomalias, agrupamento, enfim, todas essas tarefas legais de linguagem natural? Bem, eu preparei esse exemplo sobre Embeddings de palavras e agora vamos ver como tudo isso funciona. Há matemática por trás, de uma maneira que você possa entender e aplicar

## Resumo customizado

In [24]:
from langchain import LLMChain
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [25]:
template="""
    Utilize as transcrições abaixo para responder à pergunta em formato de bullet points e de forma 
    resumida. Se não souber a resposta, diga apenas que não sabe, não tente inventar ou gerar uma resposta.

    {context}

    Question: {query}
    Resposta resumida em bullter points:
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["query", "context"]
)

In [26]:
retriever = qdrant.as_retriever()

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "query": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("Resuma word embeddings")

'- Word Embeddings representam palavras e textos como vetores numéricos.\n- Palavras numericamente semelhantes têm significados semelhantes.\n- Permitem a construção de mecanismos de busca com alta precisão.\n- O processo envolve a conversão de palavras e frases em representações numéricas.\n- Utiliza modelos como textEmbeddingAda002 para gerar embeddings.\n- Os dados são armazenados em plataformas como Pinecone.\n- É possível fazer upload de dados em lotes, incluindo metadados.\n- A função getEmbedding do pacote Python da OpenAI converte texto em vetores.\n- Vetores podem ser gerados em massa para várias palavras em um DataFrame.\n- Embeddings são utilizados para encontrar semelhanças em representações numéricas.'